<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/haskell_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

行列 haskell  
で検索

1. humatrix tutorial old version   
http://dis.um.es/profesores/alberto/material/hmatrix.pdf
1, hmatrixの使い方とニューラルネットワークの実装例  
https://qiita.com/lotz/items/2c932b45f78f6fc70e9c
1. Haskellの線形代数用ライブラリで多変量ガウス分布を計算した際のメモ  
https://mmitou.hatenadiary.org/entry/20121021/1350798725
1. ENTER THE MATRIX, HASKELL STYLE  
https://www.tweag.io/blog/2017-08-31-hmatrix/
1. Dimensions and Haskell: Introduction
https://serokell.io/blog/dimensions-and-haskell-introduction
1. 行列計算用のHaskellライブラリ hmatrix を入れる  
https://mano.xyz/391/
1. hmatrix-0.20.1: Numeric Linear Algebra
1. haskell プログラム集  
https://userweb.mnet.ne.jp/tnomura/examples/matrix.html
1. Haskellで数値計算 線形代数編  
https://lqtmirage.hatenablog.com/entry/2018/07/12/232852
1. Haskellの行列計算 - 保存と読み出し、複素行列について  
https://qiita.com/tkch_pe/items/d6cfbc333d6c60751184
1. Haskell で行列の計算をしてみる  
http://tsumuji.cocolog-nifty.com/tsumuji/2011/02/haskell-0b4a.html

In [ ]:
%%capture
!apt install haskell-platform
!cabal update
!cabal install hmatrix

In [ ]:
# colab の環境では上のコマンドでオッケーだが
# 私のローカルの環境ではHaskellライブラリの他に BLAS/LAPCKというものが必要だった
# sudo apt-get install libgsl0-dev libblas-dev liblapack-dev libatlas-base-dev

In [ ]:
!ghc -e $'import Numeric.LinearAlgebra'

In [ ]:
# 上のコードでエラーが出なければ hmatrix がインストールされている
# haskell の線形代数パッケージ hmatrix は Fortran 時代から有名な
# BLAS (Basic Linear Algebra Subprograms)
# LAPACK — Linear Algebra PACKage
# を使っている

In [73]:
# hmatrix の核となるのは次の二つのデータタイプである
# data Vector e
# data Matrix e

%%writefile vector01.hs
import Numeric.LinearAlgebra
main = do
    print (vector [1,2,3] * vector [3,0,-2])
  


Overwriting vector01.hs


In [74]:
!runghc vector01.hs

[3.0,0.0,-6.0]


In [ ]:
# 見かけは list だが、型は vector である
# hmatrix> :t vector [1,2,3] * vector [3,0,-2]
# vector [1,2,3] * vector [3,0,-2] :: Vector R
#
# 要素の型の R は hmatrix の中の数値の型の synonym である
# type R = Double
# type C = Complex Double
# type I = CInt
# type Z = Int64


In [75]:
# 以上が vector だが、行列 matrix でも同様に
%%writefile matrix01.hs
import Numeric.LinearAlgebra
main = do
    print (matrix 3 [1..9] * ident 3)


Writing matrix01.hs


In [76]:
!runghc matrix01.hs

(3><3)
 [ 1.0, 0.0, 0.0
 , 0.0, 5.0, 0.0
 , 0.0, 0.0, 9.0 ]


In [ ]:
#
# hmatrix> :t matrix 3 [1..9] * ident 3
# matrix 3 [1..9] * ident 3 :: Matrix R
#
# matrix 3 は後続するリストを 3列の行列にする
# ident は 3 x 3 の単位行列を作る


In [89]:
%%writefile matrix02.hs
import Numeric.LinearAlgebra
main = do
    print (matrix 3 [1..15] )

Overwriting matrix02.hs


In [90]:
!runghc matrix02.hs

(5><3)
 [  1.0,  2.0,  3.0
 ,  4.0,  5.0,  6.0
 ,  7.0,  8.0,  9.0
 , 10.0, 11.0, 12.0
 , 13.0, 14.0, 15.0 ]


In [103]:
%%writefile matrix03.hs
import Numeric.LinearAlgebra
main = do
    print ((ident 3 ):: Matrix I)

Overwriting matrix03.hs


In [104]:
!runghc matrix03.hs

(3><3)
 [ 1, 0, 0
 , 0, 1, 0
 , 0, 0, 1 ]


In [ ]:
# GENERALISED MATRICES

In addition to the dense Vector and Matrix types, hmatrix also
 provides a ”general” matrix type GMatrix that provides optimised
 representations for dense, sparse, diagonal, banded, and constant
 matrices of Doubles . For example, the following sparse
 2x2000 matrix with two non-zero elements is represented as follows
 in the compressed sparse row (CSR) format:

hmatrix> mkSparse [((0,999),1.0),((1,1999),2.0)]
SparseR
{ gmCSR = CSR
          { csrVals = [1.0,2.0]
          , csrCols = [1000,2000]
          , csrRows = [1,2,3]
          , csrNRows = 2
          , csrNCols = 2000
          }
, nRows = 2
, nCols = 2000
}
The package includes an implementation of the conjugate gradient
 method for sparse linear systems.

FUNCTIONALITY
In addition to the basic operations on vectors and matrices,
 the core hmatrix package provides solvers for linear systems,
 computes inverses, determinants, singular value decomposition,
 eigenvalues & eigenvectors, QR, Cholesky & LU factorisation,
 and some other common matrix operations.

On top of that hmatrix-gsl covers integration, differentiation,
 FFT, solving general polynomial equations, minimization of a
 multidimensional functions, multidimensional root finding, ordinary
 differential equations, nonlinear least-squares fitting, and
 interpolation routines. Moreover, hmatrix-gsl-stats includes
 random distribution functions, linear regression functions,
 histograms, permutations, and common statistics functions (mean,
 variance, standard deviation, and so on).

Finally, hmatrix-special provides Airy, Bessel, Clausen, Coulomb
 wave, coupling coefficient, Dawson, Debye, dilogarithm, elliptic
 integral, Jacobian elliptic, Fermi-Dirac integral, Gegenbauer,
 hypergeometric, Laguerre, Lambert W, synchrotron, digamma, trigamma,
 and transport, Riemann zeta functions as well as Gamma distributions,
 Legendre polynomials, and common trigonometric and exponential
 functions.

AN EXAMPLE: MINIMA OF ARBITRARY MULTIDIMENSIONAL FUNCTIONS
For example, to find the minimum of a function


In [ ]:

f [x,y] = 10*(x-1)^2 + 20*(y-2)^2 + 30
without providing a gradient, we can define

minimizeS :: ([Double] -> Double) -> [Double] -> ([Double], Matrix
 Double)
minimizeS f xi
  = minimize NMSimplex2 1E-2 100 (replicate (length xi) 1) f
 xi
using the minimize function of hmatrix-gsl’s Numeric.GSL.Minimization.
 It provides us with the minimum as well as the path taken by
 the algorithm to reach that solution. Using gnuplot by way of
 hmatrix’ undocumented Graphics.Plot interface, to visualise
 the path, we get

Minimise Plot

For the full example code, see minimize.hs.

TYPE SAFETY
A common mistake in array programming is to apply an operation
 to one or more matrices whose dimensions violate a precondition
 of the operation. For example, for matrix multiplication to
 be well-defined for two matrices a and b, we require a to be
 ixj if b is jxk; i.e., the number of a’s columns needs to coincide
 with the number of b’s rows. Like most array libraries in Haskell,
 we cannot express such a constraint in hmatrix’ standard interface,
 where the type of a matrix is independent of the size of its
 dimensions.

The static interface of matrix provides some of hmatrix’ functionality
 with an alternative API, based on GHC’s type literals extension,
 which allows matrix size constraints to be expressed. We will
 discuss this interface in more detail in a future blog post.
CONTEXT
If you are coming from Python, then hmatrix will be the closest
 to numpy and scipy that you will find in the Haskell ecosystem.
 The packages in both languages are realised by wrapping highly
 optimised standard routines written in low-level languages 
(such as BLAS and LAPACK) as libraries. However, the functionality
 they provide is not directly comparable as, despite a strong
 overlap, both provide functionality that is absent in the other.
 While numpy and scipy shine with maturity and very widespread
 use, Haskell offers increased safety and the potential to provide
 additional high-performance functionality and fusion of computational
 kernels in Haskell itself (i.e., without the need to drop down
 to C or C++) by combining hmatrix with some of the array libraries
 that we will discuss in the forthcoming posts in this series.
